# Обработка запросов


# Обработка запросов 
- Обработка запросов - набор активностей, используемых при заборе данных из БД.
- Базовые шаги
    1. Парсинг и трансляция
    2. Оптимизация
    3. Вычисление 

До того, как будет обработан запрос, система должна транслировать запрос в подходящую форму. Внутреннее представление может различаться (обычно для общего случая рассматривается реляционная алгебра)

# Обработка запросов (схема) 

# Приведение к реляционной алгебре
Рассмотрим запрос:
```sql
SELECT salary
  FROM students 
 WHERE salary > 20000
```

Он может быть переписан на язык реляционной алгебры 

- $ \sigma_{salary>20000}(\Pi_{salary}(students)) $
- $ \Pi_{salary}(\sigma_{salary>20000}(students)) $


Более того, для каждой операции реляционной алегбры может быть выбран один из нескольких разных алгоритмов. 

В зависимости от структуры хранения и вариантов обработки данных может быть осуществлена различная выборка.

Для спецификации, каким образом будет происходить механизм обработки, необходимо не только перевести запрос
в выражение реляционной алгебры, но также и алгоритм вычисления каждой операции реляционный алгебры, причем с 
учетом количества данных и их структуры хранения. 

Операция реляционной алгебры со связанной с ней инструкцией по ее вычислению называется __примитив вычисления__ (evaluation primitive)

Последовательность примитивов вычисления, которые могут быть использованы для вычисления запроса называются __планом выполнения запроса__ (query execution plan). Механизм вычисления запроса (query execution engine) принимает на вход план выполнения запроса, выполняет его, и возвращает результат к запросу. 

Различные планы выполнения для одного и того же запроса могут иметь различную стоимость. От пользователя не ожидается, что он должен писать запросы с наиболее эффективным планом. Это задача СУБД выбрать такой план запроса, который будет минимальный по стоимости для запроса. Данная задача называется оптимизацией запросов (query optimization).

На данный момент сосредоточимся на варианте с реляционной алгеброй, другие варианты будут рассмотрены для конкретных
СУБД. 

# Меры оценки запросов


Стоимости меры оценки запросов может быть измерена в терминах различных ресурсов:

- I/O. Доступ к физическому диску 
- Время ЦП на обработку 
- Стоимость коммуникации (в параллельных и распределенных СУБД)


Для СУБД больших размеров обычно основным является вопрос I/O на дисках, однако, с учетом активного развития SSD и
In-Memory баз данных иногда возникает сильное влияние и других факторов, поэтому включают и другие значения. 

Для простоты изначально не будем включать затраты ЦП для вычисления стоимости, однако, необходимо иметь в виду, что 
в реальных СУБД учитывается гораздо больше факторов. 

Оценка запроса:

- количество переданных блоков. $ t_T$
- количество I/O произвольного доступа $t_s$ 

Примерные значения для HDD $t_s = 4$ миллисекунды и $t_T = 0.1$ миллисекунды в предположении, что размер блока - 4KB, а скорость передачи 40МБ/с. Для SSD $ t_s = 90 $ микросекунд и $ t_T $ = 4 микросекунды для 4KB блока.


Для данных, которые уже в ОП, чтение выполянет в единицах значения кеша. В предположении считывания 
всего блока $t_T < 1$ микросекунды. $ t_S < 100$ наносекунд. 

В современных СУБД значения различных базовых элементов поиска настраиваются либо автоматически при установке СУБД, либо исправляются в конфигурационном файле.

## Чтение блоков vs. Запись блоков

Обычно запись блока более чем в 2 раза дороже, по сравнению с чтением блока на магнитный диск, так как после записи происходит еще чтение блока для проверки корректности записи. 

Практически везде в дальнейших примерах запись итогового результат на диск не будет учитываться. 

## Буфер обмена 

Стоимость любого алгоритма, который будет рассматриваться далее, зависит от результата размере буфера в ОП. Лучший случай - данные всегда считывются из буфера, худший - в буфере совсем немного блоков. 

В современных СУБД худший случай является слишком пессимистичным. Обычно при расчете стоимости используется число M - количество памяти, доступное оператору, как параметр. В PostgresQL количество памяти, доступное для запроса (effective cache size) - 4 ГБ, для целей оптимизации стоимости. 

Также мы предполагаем, что данные всегда считываются с диска. Обычно довольно большое число блоков нахрдится в буфере ОП. Для учета такой возможности различные СУБД используют свои допущения: например, PostgreSQL рассматривает стоимость чтения случайной страницы как $1/10^{ой}$ от чтения страницы на диске, для моделирования ситуации о том, что 90% страниц находятся уже в буфере.

Для $B^{+}$ деревьев часто предполагают, что все внутренние вершины находятся в памяти

Время отклика для работы с планом запросов, предполагая, что нет никакой другой активности в системе, можно рассматривать как метриу оценки стоимости запроса. 

Но:

1. Время отклика зависит от содержимого буфера. Данной информации нет при оценке стоимости запроса 
2. В системе с несколькими дисками время отклика зависит от как осуществляется доступ к распеределенным дискам.

Необходимо отметить, что план может иметь лучшее время работы при увеличении используемых ресурсов. 

Вместо попытки уменьшения времени работы, оптимизатор обычно пытается минимизировать общее время потребления ресурсов плана запросов

# Операция выборки 

При обработки запросов, поиск по файлу (file scan) является самой низкоуровневой операций по забору даннух. Поиск по файлу - это алгоритмы поиска по нахождению и получению записей, которые удовлетворяют условию выборку.


## A1 Линейный поиск (linear search)

При линейном поиске осуществляется обход каждого блока файла и проверка всех записей на условие выборки. 

Сначала осуществляется поиск по файлу(переход) до первого блока. 
Если блоки файла хранятся не последовательно, то могут потребоваться 
дополнительные переходы; для простоты можно счиать, что таких переходов нет.


Линейный поиск может быть применими практически к любому файлу с 
любой структурой и физическими особенностями, но обычно они медленнее 
специфических методов

### Стоимость поиска 


$ t_s + b_r * t_T$ - 1 переход к началу файла + $b_T$ переносов блоков, где $b_t$ - число блоков

При условии поиска по первичному ключу, поиск можно прекратить после нахождения записи. Тогда в среднем будет - $ t_s + (b_r / 2) * t_T$ 

## A2 (кластерный индекс, равенство по ключу)

В случае если в таблице есть кластерный индекс, то можно использовать индекс для получения одной записи, которая удовлетворяет условию выборки. 



Стоимость запроса - $ (h_i + 1) * (t_T + t_S) $. $h_i$ - высота индекса. Происходит обход по дереву + 1 I/O для забора записи

Если внутренние вершины помещаются в буфер памяти, то $h_i = 1$

## A3 (кластерный индекс, равенство не по ключу)

При условие выборки по неключевому атрибуту, результатом может быть несколько записей. Осуществляется поиск первого блока, а затем обход по листьям

Стоимость запроса - $ h_i * ( t_T + t_S ) + t_S + b * t_T$. $b$ - 
число блоков, содержащих записи с указанным ключем поиска

## A4 (вторичный индекс по B+ дереву, равенство)

Если поиск осуществляет по ключу, то случай аналогичен A2.

В таком случае: $ (h_i + 1) * (t_T + t_S) $

Если поиск осуществляется не по ключу, то для каждой записи в худшем случае может потребоваться одна операция поиска и одна операция забора.

Стоимось - $ (h_i + n ) * (t_S + t_T) $, где $n$ - число записей

Если полноценные записи хранятся в виде $B^+$ дерева, то есть изменения. Например, вторичный индекс не хранит указатели на записи, а значение ключа. 

Также организация файла в виде $B^+$ дерева может уменьшить доступ на одну, так как сами записи уже хранятся на листе.

## A5 (кластерный индекс, сравнение)

Кластерный индекс может быть использован, если есть условие сравнения. 

Для $A<v$ или $A>=v$ осуществляется поиск значения $v$, а далее обход по листу. Для условий $A < v$ идет обход с минимального листа.

В таком случае стоимость - $h_i * (t_T + t_S) + t_S + b * t_T$

## A6 (Вторичный индекс, сравнение)

Вторичный индекс ссылается на указателя на записи, но для получения реальных значений необходима дополнительная I/O операция. 

В таком случае стоимость - $ (h_i + n ) * (t_S + t_T) $

Если корректно известно число кортежей, то оптимизатор запросов может эффективно выбрать, что использовать вторичный индекс или линейный поиск, однако, если статистика собрана не корректно, то могут быть очень большие проблемы с производительностью. 

## PostgreSQL bitmap index scan

PostgreSQL использует гибридный алгоритма под названием bitmap index scan. Изначально создается битовая карта с размером, равным количеству блоков в отношении, со всеми битами, равными 0. Затем алгоритм использует вторичный индекс, чтобы найти совпадения кортежей, но вместо забора блока данных, он делает следующее. Проихводится обход всей выборки и проставляется значение 1 в случае совпадения. После это забираются только те блоки, где было значение 1.

В хушем случае алгоритм немного более дорогой, чем линейный поиск, но в лучшем гораздо дешевле поиска по вторичному индексу. 

## Реализация сложной выборки

- Конъюнкция: $\sigma_{\theta_1 \wedge \theta_2 \wedge \dots \wedge \theta_n}(r)$
- Дизъюнкция: $\sigma_{\theta_1 \vee \theta_2 \vee \dots \vee \theta_n}(r)$
- Отрицание: $\sigma_{\neg \theta}(r)$

## A7 (Выборка конъюнкции, используя один простой индекс)


Определяем, есть ли одно из простых условий для использования в индексе. Если да, то можно выполнить один из алгоритмов A2-A6. 

Для уменьшения стоимость выбирается алгоритм для наименьшей $\theta_i$ и алгоритмов. Стоимость алгоритма совпадает с выбранными алгоритмом

## A8 (Выборка конъюнкции, используя составной индекс)

Если индекс совпадает или содержит в себе часть выборки (причем часть из первой части индекса), тогда может быть применен составной индекс. 


## A9 (Выбборка конъюнкции используя пересечение идентификаторов)

Алгоритм заключается в пересечении значений на указатели записи или их идентификаторы. В случае, если какие-то дополнительные условия остались, то осуществляется дополнительная проверка.

Стоимость - пересечение всех поисков по индексу и забор данных в отстортированным порядке.

## A10 (Выборка дизъюнкции, используя объединение идентификаторов) 

Алгоритм заключается в объединении значений на указатели записи или их идентификаторы. В случае, если какие-то дополнительные условия остались, то осуществляется требуется линейный поиск, и алгоритм становится не эффективным.

Стоимость - пересечение всех поисков по индексу и забор данных в отстортированным порядке.